In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
import requests
import datetime as dt
from scipy.stats import gompertz
from dateutil.relativedelta import *

# Create Bar Chart Race

In [3]:
df = pd.read_csv('./data_scrape.csv')
df.head()

,Tên kênh,Số lượt thích,Số lượt xem,Số bình luận,Tiêu đề,Thời lượng video,Ngày phát hành,Tag
0,Wowy,419.0,16590,10.0,(VER JBEE7) NGƯỜI VS WOWY - DUET CHALLENGE #n...,59.0,2023-11-01,NaN
1,Wowy,3786.0,164035,232.0,WOWY - NGƯỜI (M/V),249.0,2023-11-01,"['Wowy', 'vietnam', 'saigon', 'vietnamese', 'r..."
2,Wowy,85.0,2864,3.0,11.11 - MV NGƯỜI - WOWY TEASER,29.0,2023-11-01,NaN
3,Wowy,259.0,5944,5.0,Lời cảm ơn của #Wowy 🙏 sau khi bản #rap #Người...,61.0,2023-11-01,"['Wowy', 'vietnam', 'saigon', 'vietnamese', 'r..."
4,Wowy,203.0,6868,7.0,WOWY DIỄN LIVE CA KHÚC MỚI ĐẦY CHIÊM NGHIỆM #w...,61.0,2023-11-01,"['Wowy', 'vietnam', 'saigon', 'vietnamese', 'r..."


In [4]:
# Chuyển thành ngày
df['Ngày phát hành'] = df['Ngày phát hành'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))

In [5]:
channel_names = []
likes = []
views = []
title = []
date = []
c = 5
month_now = 1
year_now = 2025

for _, row in df.iterrows():
    diff_month = 12 * year_now + month_now - 12 * row['Ngày phát hành'].year - row['Ngày phát hành'].month
    if diff_month < 1:
        print(f"Skipping row with invalid date: {row['Ngày phát hành']}")
        continue
    ratios = gompertz(c).cdf(np.linspace(gompertz.ppf(0.01, c),
                                         gompertz.ppf(0.99, c), diff_month + 1))[1:]
    ratios[-1] = 1

    for i, ratio in enumerate(ratios):
        channel_names.append(row['Tên kênh'])
        likes.append(row['Số lượt thích'] * ratio)
        views.append(row['Số lượt xem'] * ratio)
        title.append(row['Tiêu đề'])
        date.append(row['Ngày phát hành'] + relativedelta(months=i+1))

new_df = pd.DataFrame(data={'Tên kênh': channel_names,
                            'Số lượt thích': likes,
                            'Số lượt xem': views,
                            'Tiêu đề': title,
                            'Ngày phát hành': date})

In [6]:
# Tỉ lệ yêu thích (Like Ratio) = (Likes / Views) * 100%
# Tỉ lệ yêu thích bình quân (Normalized Like Ratio) = (Like Ratio / Like Ratio trung bình của kênh) * 100%
new_df = new_df[new_df['Số lượt xem'] > 0]
new_df['Tỉ lệ yêu thích'] = (new_df["Số lượt thích"] / new_df["Số lượt xem"])
create_feature_dict = {'Tỉ lệ yêu thích': 'mean',
                           'Số lượt xem': 'sum',
                           'Số lượt thích': 'sum'}
runbar_df = new_df.groupby(['Tên kênh', 'Ngày phát hành']).agg(create_feature_dict).reset_index()
runbar_df['Tỉ lệ yêu thích bình quân'] = (runbar_df['Tỉ lệ yêu thích'] / runbar_df['Tỉ lệ yêu thích'].mean())
runbar_df['Ngày phát hành'] = runbar_df['Ngày phát hành'].dt.to_period('M')
runbar_df['Tỉ lệ không yêu thích bình quân'] = 1 - runbar_df['Tỉ lệ yêu thích bình quân']
runbar_df

,Tên kênh,Ngày phát hành,Tỉ lệ yêu thích,Số lượt xem,Số lượt thích,Tỉ lệ yêu thích bình quân,Tỉ lệ không yêu thích bình quân
0,AMEE,2020-05,0.007130,5.308928e+05,3541.307104,0.474552,0.525448
1,AMEE,2020-06,0.018691,1.002774e+06,7596.474563,1.243941,-0.243941
2,AMEE,2020-07,0.019906,1.571752e+06,14187.857583,1.324802,-0.324802
3,AMEE,2020-08,0.022785,2.138007e+06,21321.504891,1.516455,-0.516455
4,AMEE,2020-09,0.023273,2.685916e+06,28391.663196,1.548924,-0.548924
...,...,...,...,...,...,...,...
2805,ĐÔNG NHI,2024-09,0.016022,2.852788e+07,363753.414445,1.066313,-0.066313
2806,ĐÔNG NHI,2024-10,0.016975,2.877277e+07,366632.851231,1.129728,-0.129728
2807,ĐÔNG NHI,2024-11,0.017366,2.894835e+07,368828.547520,1.155749,-0.155749
2808,ĐÔNG NHI,2024-12,0.018083,2.909904e+07,370660.729445,1.203481,-0.203481


### Bar Chart Race - Lượt xem

In [9]:
pivot_view = pd.pivot_table(runbar_df, values='Số lượt xem', index=['Tên kênh'],
                    columns=['Ngày phát hành']).reset_index()
pivot_view.head()

Ngày phát hành,Tên kênh,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,...,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01
0,AMEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.525290e+07,1.529706e+07,1.533711e+07,1.537340e+07,1.540623e+07,1.547921e+07,1.552908e+07,1.556457e+07,1.559127e+07,1.577042e+07
1,BÍCH PHƯƠNG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.139280e+09,1.146998e+09,1.153094e+09,1.157887e+09,1.161652e+09,1.164622e+09,1.166986e+09,1.168895e+09,1.170465e+09,1.183619e+09
2,Chi Pu Official,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.630534e+08,4.649572e+08,4.694994e+08,4.729102e+08,4.754957e+08,4.774239e+08,4.788493e+08,4.804700e+08,4.814723e+08,4.870175e+08
3,Dương Domic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.064405e+06,5.944382e+06,7.332652e+06,8.388545e+06,9.467326e+06,1.071841e+07,1.878367e+07,2.247559e+07,2.984654e+07,3.160248e+07
4,ERIK Official,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.652262e+08,7.681310e+08,7.705918e+08,7.727115e+08,7.745119e+08,7.760507e+08,7.774893e+08,7.788385e+08,7.799655e+08,7.887669e+08


In [10]:
pivot_view.to_csv('./pivot_view.csv', index=False)

### Bar Chart Race - Tỉ lệ yêu thích bình quân

In [11]:
pivot_favorite = pd.pivot_table(runbar_df, values='Tỉ lệ yêu thích bình quân', index=['Tên kênh'],
                    columns=['Ngày phát hành']).reset_index()
pivot_favorite.head()

Ngày phát hành,Tên kênh,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,...,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01
0,AMEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.599858,1.599858,1.599858,1.599858,1.599858,1.772084,1.772084,1.772084,1.772084,1.772084
1,BÍCH PHƯƠNG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.278302,1.290659,1.290659,1.290659,1.290659,1.290659,1.290659,1.290659,1.290659,1.290659
2,Chi Pu Official,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.195346,1.205044,1.255935,1.258527,1.265577,1.286955,1.290214,1.494494,1.546916,1.546916
3,Dương Domic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.843492,3.825260,3.825260,3.849205,3.897659,4.094410,4.475959,4.475959,5.171516,5.171516
4,ERIK Official,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.244549,1.244549,1.244549,1.259367,1.259367,1.259367,1.279444,1.300362,1.365550,1.365550


In [12]:
pivot_favorite.to_csv('./pivot_favorite.csv', index=False)

# Bar Chart Race - Lượng người theo dõi

In [13]:
channel_df = pd.read_csv('./channel_stats.csv')
channel_df['Ngày thành lập'] = pd.to_datetime(channel_df['Ngày thành lập'])
channel_df

,Tên kênh,Số lượt đăng ký,Ngày thành lập
0,Wowy,1230000,2012-03-01
1,JustaTeeMusic,1350000,2013-02-01
2,Noo Phước Thịnh,1860000,2013-04-01
3,Trúc Nhân,1810000,2013-05-01
4,Hoàng Thùy Linh,1530000,2013-03-01
5,BÍCH PHƯƠNG,2340000,2014-11-01
6,My Tam,2450000,2013-11-01
7,ĐÔNG NHI,572000,2020-04-01
8,Sơn Tùng M-TP Official,11200000,2015-01-01
9,Hòa Minzy,1770000,2014-10-01


In [15]:
channel_names = []
subscribers = []
months_years = []
c = 5
month_now = 1
year_now = 2025

for _, row in channel_df.iterrows():
    # Tính số tháng từ ngày thành lập đến thời điểm hiện tại (hoặc thời gian dự đoán)
    diff_month = 12 * year_now + month_now - 12 * row['Ngày thành lập'].year - row['Ngày thành lập'].month
    
    if diff_month < 1:
        print(f"Skipping row with invalid date: {row['Ngày thành lập']}")
        continue
    
    # Tính toán tỷ lệ tăng trưởng dựa trên mô hình Gompertz
    ratios = gompertz(c).cdf(np.linspace(gompertz.ppf(0.01, c),
                                         gompertz.ppf(0.99, c), diff_month + 1))[1:]
    ratios[-1] = 1  # Đảm bảo tỷ lệ cuối cùng là 1 (tối đa)

    # Tạo các giá trị cho từng tháng
    for i, ratio in enumerate(ratios):
        channel_names.append(row['Tên kênh'])
        subscribers.append(row['Số lượt đăng ký'] * ratio)  # ước tính lượt đky
        month_year = (row['Ngày thành lập'] + relativedelta(months=i+1)).strftime('%Y-%m')
        months_years.append(month_year)


newChannel_df = pd.DataFrame({
    'Tên kênh': channel_names,
    'Số lượt đăng ký': subscribers,
    'Ngày thành lập': months_years
})
newChannel_df

,Tên kênh,Số lượt đăng ký,Ngày thành lập
0,Wowy,3.786612e+04,2012-04
1,Wowy,6.300035e+04,2012-05
2,Wowy,8.770777e+04,2012-06
3,Wowy,1.119934e+05,2012-07
4,Wowy,1.358623e+05,2012-08
...,...,...,...
2821,HIEUTHUHAI,1.003395e+06,2024-09
2822,HIEUTHUHAI,1.005265e+06,2024-10
2823,HIEUTHUHAI,1.006944e+06,2024-11
2824,HIEUTHUHAI,1.008451e+06,2024-12


In [16]:
pivot_subscriber = pd.pivot_table(newChannel_df, 
                                  values='Số lượt đăng ký', 
                                  index=['Tên kênh'], 
                                  columns=['Ngày thành lập'], 
                                  aggfunc='sum',  # You can adjust the aggregation function as needed (e.g., 'sum', 'mean')
                                  fill_value=0)   # Optional: Fill missing values with 0 or another value

pivot_subscriber = pivot_subscriber.reset_index()  # Reset index to make 'Tên kênh' a column
pivot_subscriber.to_csv('pivot_subscriber.csv', index = False)
